Column Details:

1. **age**: Age of the individual purchasing medical insurance.  
2. **sex**: Gender of the individual (male or female).  
3. **bmi**: Body Mass Index of the individual, a measure of body fat based on height and weight.  
4. **children**: Number of dependent children covered by the insurance.  
5. **smoker**: Smoking status of the individual (yes or no).  
6. **region**: Geographic region of the individual (e.g., northwest, southeast).  
7. **charges**: Total medical insurance premium charged to the individual.

In [2]:
#importing important libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv(r"/content/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df.shape

(1338, 7)

* We have 1334 rows and 7 columns
* Below are the names of the columns we will be working with

In [4]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [6]:
df.isnull().sum()

,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


Lukily no null vlaues.